In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
np.set_printoptions(legacy='1.13')

### Some useful functions and ranges for today

In [ ]:
thickness_bins = np.arange(-100000, 100000, 10000)
front_bins = np.arange(-150, 150, 10)
area_bins = np.arange(0, 100, 5)

In [ ]:
def sq_km_to_sq_ft(sq_km):
    return sq_km * 1195990 * 9

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

def residuals(t, x, y):
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions

## Change in thickness from year to year

In [ ]:
thickness = Table.read_table('glacier_thickness.csv')
thickness

In [ ]:
def decade(year):
    """Given a year, returns the decade it was from (e.g., 1973 -> 1970)"""
    return np.round(year, -1)

In [ ]:
decade(1973)

In [ ]:
decades = thickness.apply(decade, 'YEAR')
thickness = thickness.with_column('DECADE', decades)
thickness

In [ ]:
thickness.hist('THICKNESS_CHG', group='DECADE')

In [ ]:
two_decades = thickness.where(
    'DECADE',
    are.contained_in(make_array(1960, 2000)))
two_decades.hist('THICKNESS_CHG', group='DECADE', bins=thickness_bins)

## Glacier areas

In [ ]:
areas = Table.read_table('glacier_area.csv')
areas

In [ ]:
areas.sort('YEAR', descending=False)

In [ ]:
sq_km_to_sq_ft(4.5)

In [ ]:
areas = areas.with_column('DECADE', areas.apply(decade, 'YEAR'))
areas

In [ ]:
areas.hist('AREA', bins=area_bins)

In [ ]:
sq_km_to_sq_ft(40)

In [ ]:
# Create a table with the average area for each glacier for each decade.
# Your table should have three columns: name, decade, and the average area
# for that glacier in that decade

# NAME | YEAR | average area
# AALFOTBREEN | 1970 | ..
# AALFOTBREEN | 1980 | ...
# ...

In [ ]:
areas.drop('YEAR').group(make_array('NAME', 'DECADE'), np.average)

In [ ]:
# Create a table with the average area for each glacier for each decade.
# Your table should have three columns: name, decade, and the average area
# for that glacier in that decade


# NAME | 1980 | 2000
# AALFOTBREEN | avg area | avg area
# AALFOTBREEN | 1980 | ...
# ...

In [ ]:

by_name_and_decade = areas.drop('YEAR').group(make_array('NAME', 'DECADE'), np.mean)
by_name_and_decade

In [ ]:
avg_1980s = (
    by_name_and_decade
    .where('DECADE', 1980)
    .drop('DECADE')
    .relabeled(1, '1980')
)
avg_2000s = by_name_and_decade.where('DECADE', 2000).drop('DECADE').relabeled(1, '2000')
avg_1980s

In [ ]:
comparison = avg_1980s.join('NAME', avg_2000s)

In [ ]:
comparison

In [ ]:
sq_km_to_sq_ft(1500)

In [ ]:
comparison.scatter('1980', '2000')

In [ ]:
correlation(comparison, '1980', '2000')

In [ ]:
slope(comparison, '1980', '2000')

In [ ]:
comparison.with_column('Residuals', residuals(comparison, '1980', '2000')).scatter('1980', 'Residuals')

In [ ]:
comparison = comparison.where('1980', are.below(100))

In [ ]:
comparison.scatter('1980', '2000')

In [ ]:
correlation(comparison, '1980', '2000')

In [ ]:
slope(comparison, '1980', '2000')

In [ ]:
intercept(comparison, '1980', '2000')

In [ ]:
comparison.with_column('Residuals', residuals(comparison, '1980', '2000')).scatter('1980', 'Residuals')

In [ ]:
comparison = comparison.with_column(
    'Change',
    comparison.column('2000') - comparison.column('1980')
)

In [ ]:
comparison

In [ ]:
comparison.hist('Change')

In [ ]:
comparison.hist('Change', bins=np.arange(-5, 5, .5))

In [ ]:
np.median(comparison.column('Change'))

In [ ]:
np.mean(comparison.column('Change'))

In [ ]:
np.median(comparison.column('Change'))

In [ ]:
sq_km_to_sq_ft(np.median(comparison.column('Change')))

In [ ]:
def bootstrap_median(original_sample, label, replications):
    """Returns an array of bootstrapped sample medians:
    original_sample: table containing the original sample
    label: label of column containing the variable
    replications: number of bootstrap samples
    """
    just_one_column = original_sample.select(label)
    medians = make_array()
    for i in np.arange(replications):
        bootstrap_sample = just_one_column.sample()
        resampled_median = percentile(50, bootstrap_sample.column(0))
        medians = np.append(medians, resampled_median)
        
    return medians

In [ ]:
medians = bootstrap_median(comparison, 'Change', 10000)
medians_in_sq_ft = sq_km_to_sq_ft(medians)

In [ ]:
left = percentile(2.5, medians_in_sq_ft)
right = percentile(97.5, medians_in_sq_ft)

In [ ]:

Table().with_column('Bootstrap Medians', medians_in_sq_ft).hist('Bootstrap Medians')
plots.plot([left, right], [0,0], color="gold",lw=3, zorder=1);
plots.scatter(np.median(sq_km_to_sq_ft(comparison.column('Change'))), 0, color="blue", zorder=2);

## Change in glacier fronts: receding ice

In [ ]:
front = Table.read_table('glacier_front.csv')
front

In [ ]:
decades = front.apply(decade, 'YEAR')
front = front.with_column('DECADE', decades)
front

In [ ]:
front.hist('FRONT_VARIATION', bins=front_bins)

In [ ]:
recent = front.where('DECADE', are.above(1980))
recent.hist('FRONT_VARIATION', bins=front_bins)

In [ ]:
# Create a table with the average front variation for each glacier for each decade.
# Your table should have three columns: name, decade, and the average FRONT_VARIATION
# for that glacier in that decade

by_name_and_decade = front.drop('YEAR').group(make_array('NAME', 'DECADE'), np.mean)
by_name_and_decade

In [ ]:
avg_1980s = by_name_and_decade.where('DECADE', 1980).drop('DECADE').relabeled(1, '1980')
avg_2000s = by_name_and_decade.where('DECADE', 2000).drop('DECADE').relabeled(1, '2000')
comparison = avg_1980s.join('NAME', avg_2000s)

In [ ]:
comparison.scatter('1980', '2000')

In [ ]:
comparison.where('1980', are.above(-500)).where('2000', are.above(-500)).scatter('1980', '2000')